In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
import sys
from datetime import datetime
import traceback

In [2]:
# Create a set of random latitude and longitude combinations.
num_lats = 1500
lats = np.random.uniform(low=-90.000, high=90.000, size=num_lats)
lngs = np.random.uniform(low=-180.000, high=180.000, size=num_lats)
lat_lngs = zip(lats, lngs)
len(lats) 

1500

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
'''
{'coord': {'lon': 20.04, 'lat': -34.53}, 
'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 
'base': 'stations', 
'main': {'temp': 68.49, 'feels_like': 65.84, 'temp_min': 68, 'temp_max': 69.01, 'pressure': 1014, 'humidity': 60}, 
'wind': {'speed': 6.93, 'deg': 160}, 
'clouds': {'all': 0}, 
'dt': 1587227625, 
'sys': {'type': 1, 'id': 1966, 'country': 'ZA', 'sunrise': 1587186325, 'sunset': 1587226343}, 
'timezone': 7200, 'id': 1015776, 'name': 'Bredasdorp', 'cod': 200}
'''
def create_city_url(c_name):
    # Starting URL for Weather Map API Call.
    units = "Imperial"
    api_url = "http://api.openweathermap.org/data/2.5/weather"
    city_url = f"{api_url}?units={units}&APPID={weather_api_key}&q={c_name}"
    return city_url

In [8]:
weather_list=[]
for coordinate in coordinates:
    nearest_city = citipy.nearest_city(coordinate[0], coordinate[1])
    c_name = nearest_city.city_name
    c_country_code = nearest_city.country_code
    city_url = create_city_url(c_name)
    try:
        print(f"Processing city: {c_name}")
        city_weather = requests.get(city_url)
        if city_weather.status_code == 200:
            city_weather_json = city_weather.json()
            
            lat = city_weather_json["coord"]["lat"]
            lng = city_weather_json["coord"]["lon"]
            max_temp = city_weather_json["main"]["temp_max"]
            humidity = city_weather_json["main"]["humidity"]
            cloudiness = city_weather_json["clouds"]["all"]
            wind_speed = city_weather_json["wind"]["speed"]
            description = city_weather_json["weather"][0]["description"]
            dt_object = datetime.fromtimestamp(city_weather_json["dt"])
            date_time = dt_object.strftime("%m/%d/%Y, %H:%M:%S")
            try:
                city_rain = city_weather_json["rain"]["3h"]
            except (KeyError,NameError) as e:
                city_rain = 0
                print(f"Error: Rain processing city {c_name}: Exception: {str(e)}")
                pass
            try:
                city_snow = city_weather_json["snow"]["3h"]
                print(f"Found snow: {city_snow}")
            except (KeyError,NameError) as e:
                city_snow = 0
                print(f"Key Error: Snow processing city: {c_name}: Exception: {str(e)}")
                pass
            weather_dict = {
                "City": c_name,
                "Country": c_country_code,
                "Date": date_time,
                "Lat": lat,
                "Lng": lng,
                "Max Temp": max_temp,
                "Humidity": humidity,
                "Cloudiness": cloudiness,
                "Wind Speed": wind_speed,
                "Current Description": description,
                "Rain inches (last 3 hrs)": city_rain,
                "Snow inches (last 3 hrs)": city_snow
            }
            weather_list.append(weather_dict)
        else:
            print(f"City Weather not found city: {c_name}")
    except AttributeError as ae:
        print(f"Error processing city: {c_name} AttributeError:{str(ae)}")
    except Exception as e:
        print(f"Error processing city {c_name} API Request:{str(e)}")

Processing city: walvis bay
Error: Rain processing city walvis bay: Exception: 'rain'
Key Error: Snow processing city: walvis bay: Exception: 'snow'
Processing city: avarua
Error: Rain processing city avarua: Exception: 'rain'
Key Error: Snow processing city: avarua: Exception: 'snow'
Processing city: osa
Error: Rain processing city osa: Exception: 'rain'
Key Error: Snow processing city: osa: Exception: 'snow'
Processing city: port elizabeth
Error: Rain processing city port elizabeth: Exception: 'rain'
Key Error: Snow processing city: port elizabeth: Exception: 'snow'
Processing city: bredasdorp
Error: Rain processing city bredasdorp: Exception: 'rain'
Key Error: Snow processing city: bredasdorp: Exception: 'snow'
Processing city: puerto ayora
Key Error: Snow processing city: puerto ayora: Exception: 'snow'
Processing city: ponta do sol
Error: Rain processing city ponta do sol: Exception: 'rain'
Key Error: Snow processing city: ponta do sol: Exception: 'snow'
Processing city: quchan
Er

Error: Rain processing city lebu: Exception: 'rain'
Key Error: Snow processing city: lebu: Exception: 'snow'
Processing city: boguchany
Error: Rain processing city boguchany: Exception: 'rain'
Key Error: Snow processing city: boguchany: Exception: 'snow'
Processing city: martos
Key Error: Snow processing city: martos: Exception: 'snow'
Processing city: cayenne
Error: Rain processing city cayenne: Exception: 'rain'
Key Error: Snow processing city: cayenne: Exception: 'snow'
Processing city: fazilka
Error: Rain processing city fazilka: Exception: 'rain'
Key Error: Snow processing city: fazilka: Exception: 'snow'
Processing city: rikitea
Error: Rain processing city rikitea: Exception: 'rain'
Key Error: Snow processing city: rikitea: Exception: 'snow'
Processing city: pedernales
Error: Rain processing city pedernales: Exception: 'rain'
Key Error: Snow processing city: pedernales: Exception: 'snow'
Processing city: castro
Error: Rain processing city castro: Exception: 'rain'
Key Error: Snow

Error: Rain processing city daman: Exception: 'rain'
Key Error: Snow processing city: daman: Exception: 'snow'
Processing city: arraial do cabo
Error: Rain processing city arraial do cabo: Exception: 'rain'
Key Error: Snow processing city: arraial do cabo: Exception: 'snow'
Processing city: guerrero negro
Error: Rain processing city guerrero negro: Exception: 'rain'
Key Error: Snow processing city: guerrero negro: Exception: 'snow'
Processing city: tasiilaq
Error: Rain processing city tasiilaq: Exception: 'rain'
Key Error: Snow processing city: tasiilaq: Exception: 'snow'
Processing city: acarau
Error: Rain processing city acarau: Exception: 'rain'
Key Error: Snow processing city: acarau: Exception: 'snow'
Processing city: ushuaia
Error: Rain processing city ushuaia: Exception: 'rain'
Key Error: Snow processing city: ushuaia: Exception: 'snow'
Processing city: ostersund
Error: Rain processing city ostersund: Exception: 'rain'
Key Error: Snow processing city: ostersund: Exception: 'snow

Error: Rain processing city goderich: Exception: 'rain'
Key Error: Snow processing city: goderich: Exception: 'snow'
Processing city: busselton
Key Error: Snow processing city: busselton: Exception: 'snow'
Processing city: ushuaia
Error: Rain processing city ushuaia: Exception: 'rain'
Key Error: Snow processing city: ushuaia: Exception: 'snow'
Processing city: punta arenas
Error: Rain processing city punta arenas: Exception: 'rain'
Key Error: Snow processing city: punta arenas: Exception: 'snow'
Processing city: rikitea
Error: Rain processing city rikitea: Exception: 'rain'
Key Error: Snow processing city: rikitea: Exception: 'snow'
Processing city: tilichiki
Error: Rain processing city tilichiki: Exception: 'rain'
Key Error: Snow processing city: tilichiki: Exception: 'snow'
Processing city: ushuaia
Error: Rain processing city ushuaia: Exception: 'rain'
Key Error: Snow processing city: ushuaia: Exception: 'snow'
Processing city: mundi
Error: Rain processing city mundi: Exception: 'rai

Error: Rain processing city abha: Exception: 'rain'
Key Error: Snow processing city: abha: Exception: 'snow'
Processing city: mitsamiouli
Key Error: Snow processing city: mitsamiouli: Exception: 'snow'
Processing city: new norfolk
Error: Rain processing city new norfolk: Exception: 'rain'
Key Error: Snow processing city: new norfolk: Exception: 'snow'
Processing city: busselton
Key Error: Snow processing city: busselton: Exception: 'snow'
Processing city: karratha
Error: Rain processing city karratha: Exception: 'rain'
Key Error: Snow processing city: karratha: Exception: 'snow'
Processing city: barrow
Error: Rain processing city barrow: Exception: 'rain'
Key Error: Snow processing city: barrow: Exception: 'snow'
Processing city: taolanaro
City Weather not found city: taolanaro
Processing city: hobart
Error: Rain processing city hobart: Exception: 'rain'
Key Error: Snow processing city: hobart: Exception: 'snow'
Processing city: kiunga
Error: Rain processing city kiunga: Exception: 'ra

...

Key Error: Snow processing city: ribeira grande: Exception: 'snow'
Processing city: carballo
Error: Rain processing city carballo: Exception: 'rain'
Key Error: Snow processing city: carballo: Exception: 'snow'
Processing city: albany
Error: Rain processing city albany: Exception: 'rain'
Key Error: Snow processing city: albany: Exception: 'snow'
Processing city: airai
Error: Rain processing city airai: Exception: 'rain'
Key Error: Snow processing city: airai: Exception: 'snow'
Processing city: bluff
Key Error: Snow processing city: bluff: Exception: 'snow'
Processing city: valparaiso
Error: Rain processing city valparaiso: Exception: 'rain'
Key Error: Snow processing city: valparaiso: Exception: 'snow'
Processing city: nizhneyansk
City Weather not found city: nizhneyansk
Processing city: mataura
Key Error: Snow processing city: mataura: Exception: 'snow'
Processing city: arraial do cabo
Error: Rain processing city arraial do cabo: Exception: 'rain'
Key Error: Snow processing city: arrai

Error: Rain processing city bambous virieux: Exception: 'rain'
Key Error: Snow processing city: bambous virieux: Exception: 'snow'
Processing city: chuy
Error: Rain processing city chuy: Exception: 'rain'
Key Error: Snow processing city: chuy: Exception: 'snow'
Processing city: jamestown
Error: Rain processing city jamestown: Exception: 'rain'
Key Error: Snow processing city: jamestown: Exception: 'snow'
Processing city: havre-saint-pierre
Error: Rain processing city havre-saint-pierre: Exception: 'rain'
Key Error: Snow processing city: havre-saint-pierre: Exception: 'snow'
Processing city: barentsburg
City Weather not found city: barentsburg
Processing city: havre-saint-pierre
Error: Rain processing city havre-saint-pierre: Exception: 'rain'
Key Error: Snow processing city: havre-saint-pierre: Exception: 'snow'
Processing city: yandyki
Error: Rain processing city yandyki: Exception: 'rain'
Key Error: Snow processing city: yandyki: Exception: 'snow'
Processing city: ushuaia
Error: Rain

Key Error: Snow processing city: bluff: Exception: 'snow'
Processing city: bolshaya murta
City Weather not found city: bolshaya murta
Processing city: kruisfontein
Error: Rain processing city kruisfontein: Exception: 'rain'
Key Error: Snow processing city: kruisfontein: Exception: 'snow'
Processing city: alice springs
Error: Rain processing city alice springs: Exception: 'rain'
Key Error: Snow processing city: alice springs: Exception: 'snow'
Processing city: buren
Key Error: Snow processing city: buren: Exception: 'snow'
Processing city: mataura
Key Error: Snow processing city: mataura: Exception: 'snow'
Processing city: new norfolk
Error: Rain processing city new norfolk: Exception: 'rain'
Key Error: Snow processing city: new norfolk: Exception: 'snow'
Processing city: faya
Error: Rain processing city faya: Exception: 'rain'
Key Error: Snow processing city: faya: Exception: 'snow'
Processing city: mataura
Key Error: Snow processing city: mataura: Exception: 'snow'
Processing city: ca

Error: Rain processing city tuktoyaktuk: Exception: 'rain'
Key Error: Snow processing city: tuktoyaktuk: Exception: 'snow'
Processing city: ushuaia
Error: Rain processing city ushuaia: Exception: 'rain'
Key Error: Snow processing city: ushuaia: Exception: 'snow'
Processing city: pinawa
Error: Rain processing city pinawa: Exception: 'rain'
Found snow: 0.12
Processing city: bethel
Error: Rain processing city bethel: Exception: 'rain'
Key Error: Snow processing city: bethel: Exception: 'snow'
Processing city: avarua
Error: Rain processing city avarua: Exception: 'rain'
Key Error: Snow processing city: avarua: Exception: 'snow'
Processing city: marsh harbour
Error: Rain processing city marsh harbour: Exception: 'rain'
Key Error: Snow processing city: marsh harbour: Exception: 'snow'
Processing city: howard springs
Error: Rain processing city howard springs: Exception: 'rain'
Key Error: Snow processing city: howard springs: Exception: 'snow'
Processing city: lorengau
Key Error: Snow process

Error: Rain processing city fortuna: Exception: 'rain'
Key Error: Snow processing city: fortuna: Exception: 'snow'
Processing city: fortuna
Error: Rain processing city fortuna: Exception: 'rain'
Key Error: Snow processing city: fortuna: Exception: 'snow'
Processing city: margate
Error: Rain processing city margate: Exception: 'rain'
Key Error: Snow processing city: margate: Exception: 'snow'
Processing city: yellowknife
Error: Rain processing city yellowknife: Exception: 'rain'
Key Error: Snow processing city: yellowknife: Exception: 'snow'
Processing city: lavrentiya
Error: Rain processing city lavrentiya: Exception: 'rain'
Found snow: 0.53
Processing city: baracoa
Error: Rain processing city baracoa: Exception: 'rain'
Key Error: Snow processing city: baracoa: Exception: 'snow'
Processing city: ushuaia
Error: Rain processing city ushuaia: Exception: 'rain'
Key Error: Snow processing city: ushuaia: Exception: 'snow'
Processing city: tumannyy
City Weather not found city: tumannyy
Proces

In [6]:
weather_df = pd.DataFrame(weather_list)
weather_df.head(20)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,walvis bay,na,"04/18/2020, 18:58:15",-22.96,14.51,55.99,93,2,2.24,clear sky,0.00,0.0
1,avarua,ck,"04/18/2020, 18:56:26",-21.21,-159.78,82.40,65,79,9.17,broken clouds,0.00,0.0
2,osa,ru,"04/18/2020, 18:58:15",57.28,55.46,36.19,80,22,8.34,few clouds,0.00,0.0
3,port elizabeth,za,"04/18/2020, 18:55:17",-33.92,25.57,59.00,100,0,2.24,clear sky,0.00,0.0
4,bredasdorp,za,"04/18/2020, 18:58:15",-34.53,20.04,48.20,93,94,5.82,overcast clouds,0.00,0.0
5,puerto ayora,ec,"04/18/2020, 18:56:22",-0.74,-90.35,80.01,88,10,4.00,light rain,1.13,0.0
6,ponta do sol,cv,"04/18/2020, 18:58:16",32.67,-17.10,57.00,63,4,5.99,clear sky,0.00,0.0
7,quchan,ir,"04/18/2020, 18:58:16",37.11,58.51,52.68,38,0,3.42,clear sky,0.00,0.0
8,salta,ar,"04/18/2020, 18:58:16",-24.79,-65.41,59.00,87,0,4.70,clear sky,0.00,0.0
9,mataura,pf,"04/18/2020, 18:55:13",-46.19,168.86,54.00,70,64,13.00,light rain,1.33,0.0


In [9]:
weather_df.to_csv (r'./analysis/WeatherPy_challenge.csv', index = False, header=True)

In [12]:
len(weather_df)

1379

In [13]:
len(weather_df[weather_df["Rain inches (last 3 hrs)"]>0])

235

In [14]:
len(weather_df[weather_df["Snow inches (last 3 hrs)"]>0])

33